In [1]:
# Import Libraries

import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')


# Import dataset

df = pd.read_excel('Book2.xlsx')

In [2]:
# df.columns
# Clean the data
    ## Choose neccessary columns : 'Mã học phần','Tên học phần', 'Mã lớp tín chỉ', 'Số tín chỉ','Giai đoạn', 'Thứ', 'Tiết bắt đầu', 'LỊCH HỌC', 'Mã lớp'
    ##

In [3]:
cols = ['Mã học phần','Tên học phần', 'Mã lớp tín chỉ', 'Số tín chỉ','Giai đoạn', 'Thứ', 'Tiết bắt đầu', 'LỊCH HỌC', 'Mã lớp']
df_clean = df[cols]

In [4]:
df_clean.rename(columns={'Mã học phần': 'course_code', 'Tên học phần': 'course_name', 'Mã lớp tín chỉ': 'class_code',
                   'Số tín chỉ': 'num_of_credits', 'Giai đoạn': 'period', 'Thứ': 'weekday', 'Tiết bắt đầu': 'start',
                   'LỊCH HỌC': 'time', 'Mã lớp': 'code'}, inplace=True)

In [5]:
df_clean = df_clean[~df_clean['time'].isnull()]

In [6]:
df_clean = df_clean[~df_clean['time'].isnull()]

list_blank = ['Thứ 2(01-03)', 'Thứ 2(04-06)', 'Thứ 2(10-12)', 'Thứ 3(01-03)','Thứ 2(07-09)']
df_clean['weekday'][df_clean['weekday'].isnull()] = df_clean['time'].map(lambda x: '2Ca1' if x == list_blank[0] else (
                                                '2Ca2' if x == list_blank[1] else(
                                                '2Ca4' if x == list_blank[2] else(
                                                '2Ca4' if x == list_blank[4] else '3Ca1'
                                                )
                                                )
                                                )
                                            )

In [7]:
df_clean['period_weekday'] =  df_clean['weekday'] + '.' + df_clean['period'].astype(str)
df_clean = df_clean[df_clean['period'].isin([1,2])]
df_clean['short_code'] = df_clean['course_code'] + '.' + df_clean['code'].astype(str) + '.' + df_clean['period'].astype(str)

In [8]:
import itertools
list_subjects = ['MKT407', 'KTE312', 'TMA404', 'KTE319', 'TMA311']
list_set = []
for i in list_subjects:
    set = df_clean['short_code'][df_clean['course_code']==i].unique().tolist()
    list_set.append(set)

all_list = [] 
for element in itertools.product(*list_set):
  all_list.append(element)

In [9]:
full_list=[]
for list in all_list:
      xls = pd.ExcelFile('Timetable Project.xlsx')
      df_main= pd.read_excel(xls, 'main')
      for x in list:
            
            list_slots = df_clean['period_weekday'][df_clean['short_code']==x].unique().tolist()
            df_main[x] = 0
            df_main[x] = df_main['Ca'].map(lambda x: 1 if x in list_slots else 0)

      df_main['Sum'] = df_main.sum(axis=1)
      
      if (df_main['Sum']<=1).all():
            full_list.append(list)
      else:
            pass

In [10]:
for list in full_list:
    xls = pd.ExcelFile('Timetable Project.xlsx')
    df_output= pd.read_excel(xls, 'output')
    for x in list:

        ca_list = df_clean['period_weekday'][df_clean['short_code'] == x]
        for ca in ca_list:
            day = int(ca[0])
            slot = ca[1:]
            df_output[day][df_output['Ca']==slot] = x

    print(df_output)
    print('==============================================================================')

      Ca           2           3           4           5           6
0  Ca1.1  TMA404.1.1  MKT407.2.1         NaN  TMA311.4.1         NaN
1  Ca2.1  TMA311.4.1         NaN  TMA404.1.1  MKT407.2.1         NaN
2  Ca3.1         NaN  KTE312.1.1         NaN         NaN  KTE319.1.1
3  Ca4.1         NaN  KTE319.1.1         NaN  KTE312.1.1         NaN
4    sep         sep         sep         sep         sep         sep
5  Ca1.2         NaN         NaN         NaN         NaN         NaN
6  Ca2.2         NaN         NaN         NaN         NaN         NaN
7  Ca3.2         NaN         NaN         NaN         NaN         NaN
8  Ca4.2         NaN         NaN         NaN         NaN         NaN
      Ca           2           3           4           5           6
0  Ca1.1  TMA404.1.1  MKT407.2.1         NaN         NaN         NaN
1  Ca2.1         NaN         NaN  TMA404.1.1  MKT407.2.1         NaN
2  Ca3.1         NaN  KTE312.1.1  TMA311.6.1         NaN  KTE319.1.1
3  Ca4.1         NaN  KTE319.1.1  